In [1]:
import findspark
findspark.init('/home/gani/spark-2.4.4-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('missingVal').getOrCreate()

In [4]:
df = spark.read.csv('ContainsNull.csv', inferSchema = True, header = True)

In [5]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [6]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [8]:
df.na.drop().show()
#df.dropna().show() #alternatively

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [10]:
#df.na.drop(thresh = 2).show()
df.dropna(thresh=2).show() #alternatively

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [16]:
#df.na.drop(how='any').show()
#df.na.drop(how='all').show()
#df.dropna(how='any').show() #alternatively
df.dropna(how='all').show() #alternatively

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [19]:
#df.na.drop(subset=['Sales']).show()
df.dropna(subset=['Sales']).show() #alternatively

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [21]:
#df.na.fill('Fill Val').show()
df.fillna('Fill Val').show() #alternatively

+----+--------+-----+
|  Id|    Name|Sales|
+----+--------+-----+
|emp1|    John| null|
|emp2|Fill Val| null|
|emp3|Fill Val|345.0|
|emp4|   Cindy|456.0|
+----+--------+-----+



In [23]:
#df.na.fill(0).show()
df.fillna(0).show() #alternatively

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [25]:
#df.na.fill('No Name',subset=['Name']).show()
df.fillna('No Name', subset='Name').show() #alternatively

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|No Name| null|
|emp3|No Name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



In [26]:
from pyspark.sql.functions import mean

In [27]:
mean_val = df.select(mean(df['Sales'])).collect()
mean_val

[Row(avg(Sales)=400.5)]

In [29]:
#mean_val[0]
mean_val[0][0]

400.5

In [35]:
#alternatively
row = mean_val[0]
#row[0]
mean_sales = row.asDict()['avg(Sales)'] #here column name is avg(Sales), you could use alias for simpler col name

In [36]:
df.na.fill(mean_sales, ['Sales']).show() #not explicitly mentioning subset but still works as its in correct order

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [38]:
#1 line
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0], ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

